## Deliverable 2. Create a Customer Travel Destinations Map.

In [37]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('/Users/RosieMina/Desktop/Analysis Projects/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv')
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ribeira Grande,PT,38.5167,-28.7000,64.80,100,100,8.05,mist
1,1,Hobyo,SO,5.3505,48.5268,80.40,79,100,26.04,overcast clouds
2,2,Tukrah,LY,32.5341,20.5791,64.18,91,0,6.82,clear sky
3,3,Sitka,US,57.0531,-135.3300,61.45,52,1,5.23,clear sky
4,4,Sao Joao Da Barra,BR,-21.6403,-41.0511,73.49,73,9,10.87,clear sky


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 85
What is the maximum temperature you would like for your trip? 93


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [35]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = clean_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
13,13,Yarada,IN,17.6500,83.2667,86.32,74,75,9.22,mist
35,35,Half Moon Bay,US,37.4636,-122.4286,86.88,51,0,16.11,clear sky
48,48,Albany,US,42.6001,-73.9662,90.77,54,4,1.01,clear sky
84,84,Ixtapa,MX,20.7000,-105.2000,92.75,36,1,8.50,clear sky
98,98,Mecca,SA,21.4267,39.8261,88.30,25,4,0.49,clear sky


In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Yarada,IN,86.32,mist,17.6500,83.2667,
35,Half Moon Bay,US,86.88,clear sky,37.4636,-122.4286,
48,Albany,US,90.77,clear sky,42.6001,-73.9662,
84,Ixtapa,MX,92.75,clear sky,20.7000,-105.2000,
98,Mecca,SA,88.30,clear sky,21.4267,39.8261,
116,Buraydah,SA,91.85,clear sky,26.3260,43.9750,
155,Ukiah,US,85.44,scattered clouds,39.1502,-123.2078,
177,Atar,MR,92.68,clear sky,20.5169,-13.0499,
228,Broome,US,91.60,scattered clouds,42.2506,-75.8330,
237,Sur,OM,90.05,clear sky,22.5667,59.5289,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    ""
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (IndexError):
        print("No Hotel Found. Skipping...")

No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...
No Hotel Found. Skipping...


In [38]:
# 7. Drop the rows where there is no Hotel Name.
clean_df = hotel_df.replace('', np.nan, regex=True)
clean_hotel_df = clean_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Yarada,IN,86.32,mist,17.6500,83.2667,Yarada Jungle Beach Resorts
35,Half Moon Bay,US,86.88,clear sky,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay"
84,Ixtapa,MX,92.75,clear sky,20.7000,-105.2000,hotel la terminal
98,Mecca,SA,88.30,clear sky,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
116,Buraydah,SA,91.85,clear sky,26.3260,43.9750,Mövenpick Qassim
155,Ukiah,US,85.44,scattered clouds,39.1502,-123.2078,Hampton Inn Ukiah
177,Atar,MR,92.68,clear sky,20.5169,-13.0499,Oumou Elghoura
228,Broome,US,91.60,scattered clouds,42.2506,-75.8330,Chenango Valley State Park
237,Sur,OM,90.05,clear sky,22.5667,59.5289,Sur Plaza Hotel
241,San Ramon,US,87.94,clear sky,37.7799,-121.9780,San Ramon Marriott


In [40]:
# 8a. Create the output File (CSV)
output_data_file = '../Vacation_Search/WeatherPy_Vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [42]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
figWearhe

Figure(layout=FigureLayout(height='420px'))